In [119]:
# imports
import tensorflow as tf
from tensorflow.contrib.layers import xavier_initializer as xav
import numpy as np

In [120]:
class controller(object):
    def __init__(self, layers):
        self.sess = tf.Session()
        self.layers = layers

        with tf.variable_scope("weights", reuse=None):
            prev_size = layers[0]
            cur_layer = 1
            self.weights = []
            xav_init = xav(dtype=tf.float32)
            for size in layers[1:]:
                # self.weights.append(xav_init(shape = [size, prev_size]))
                self.weights.append(tf.get_variable(name="w" + str(cur_layer), shape=[size, prev_size],
                                                    initializer=xav_init))
                prev_size = size
                cur_layer += 1

        # self.weights = self.weights_initialize()
        self.optimizer = tf.train.AdamOptimizer()

    def train(self, prediction, target):

        loss = tf.reduce_sum(tf.square(prediction - target))
        loss_step = self.optimizer.minimize(loss)

        return loss, loss_step

    # def weights_initialize(self):
    #     layers = self.layers
    # 
    #     weights = []
    #     prev_size = layers[0]
    # 
    #     xav_init = tf.contrib.layers.xavier_initializer(dtype = tf.float32)
    #     for size in layers[1:]:
    #         weights.append(self.sess.run(xav_init(shape = [size, prev_size])))
    #         prev_size = size
    # 
    #     return weights

    def forward(self, input):

        layer = input

        with tf.variable_scope("weights", reuse=True):
            for layer_index in range(len(self.layers) - 1):
                layer = tf.matmul(self.weights[layer_index], layer)
                if layer_index != len(self.layers) - 2:
                    layer = tf.nn.relu(layer)

        return layer

In [121]:
class NTM(object):
    """
    Controller config
    Memory matrix
    Read vectors / weights
    
    """
    def __init__(self, layers, mem_size):
        self.controller = controller(layers)
        with tf.variable_scope("ntm", reuse=None):
            self.memory = tf.zeros(shape=[mem_size, mem_size], dtype=tf.float32, name="memory")
            self.read_vector = tf.zeros(mem_size)
            xav_init = xav(dtype=tf.float32)
            self.weights = tf.get_variable("weights", initializer=xav_init, shape=[mem_size, 1])


    def read(self):
        with tf.variable_scope("ntm", reuse=True):
            mem = self.memory
            wts = self.weights
            self.read_vector = tf.matmul(mem, wts)


    def write(self):
        pass

In [122]:
with tf.Graph().as_default():
    ntm = NTM([100,200,300,200,40], 5)
    ntm.read()
    # wts = ntm.controller.weights
    # wts = tf.Session().run(wts)
    # for k in wts:
    #     print(np.shape(k))
    sess = tf.Session()

    # input = np.random.rand(100,1).astype(dtype=np.float32)
    # pred = ntm.controller.forward(input)
    # # pred1 = tf.Session().run(pred)
    
    writer = tf.summary.FileWriter("/Users/shravan/codes/NTM/logs")
    writer.add_graph(sess.graph)
       
    sess.run(tf.global_variables_initializer())
    read_vec = sess.run(ntm.read_vector)
    print(read_vec)
    # 
    # target = np.zeros([40,1])
    # target[4] = 1
    
    # 
    # loss, train_step = ntm.controller.train(pred, target)
    # sess.run(tf.global_variables_initializer())
    # for _ in range(100):
    #     ls, _ = sess.run([loss, train_step])
    #     print(ls)
    # print(" new ")
    # for _ in range(10):
    #     ls, _ = sess.run([loss, train_step])
    #     print(ls)
    # print(sess.run(pred), sess.run(pred))

[[ 0.]
 [ 0.]
 [ 0.]
 [ 0.]
 [ 0.]]
